In [7]:
import logging
logging.basicConfig(format='%(asctime)s : %(threadName)s : %(levelname)s : %(message)s', level=logging.INFO)

import gensim.downloader as api
data = api.load("quora-duplicate-questions")
#w2v = api.load("glove-wiki-gigaword-100")

from fse import IndexedList
from fse.models.average import MAX_WORDS_IN_BATCH, Average
MAX_WORDS_IN_BATCH
#w2v_avg = Average(w2v)

10000

In [8]:
sentences = []
cum = 0
for i, d in enumerate(data):
    sent = d["question1"].split()
    cum += len(sent)
    if cum <= MAX_WORDS_IN_BATCH:
        sentences.append(sent)
    else:
        break
sents = IndexedList(sentences, pre_splitted=True)
print(len(sents))

933


In [9]:
%%timeit
for i, s in enumerate(sents):
    _ = (s, i)

755 µs ± 54.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [10]:
%%timeit
for i, s in enumerate(sents):
    _ = IndexedSentence(s, i)

1.01 ms ± 36.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [15]:
import joblib
sentences = joblib.load("sentences.joblib")
sents = IndexedList(sentences, pre_splitted=True)

In [16]:
%%timeit
for s in sents:
    _ = s[0]

135 ms ± 11.2 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [17]:
len(sents)

404290

In [3]:
from fse.models.average import Average, train_average_np
from fse.models.average_inner import train_average_cy

In [4]:
statistics = w2v_avg.scan_sentences(sents)
w2v_avg.prep.prepare_vectors(sv=w2v_avg.sv, total_sentences=statistics["max_index"], update=False)
memory = w2v_avg._get_thread_working_mem()

2019-09-04 18:48:52,035 : MainThread : INFO : scanning all indexed sentences and their word counts
2019-09-04 18:48:52,038 : MainThread : INFO : finished scanning 933 sentences with an average length of 10 and 9986 total words
2019-09-04 18:48:52,040 : MainThread : INFO : initializing sentence vectors for 933 sentences


In [5]:
%%timeit
train_average_cy(model=w2v_avg, indexed_sentences=sents, target=w2v_avg.sv.vectors, memory=memory)

2.69 ms ± 364 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
